<a href="https://colab.research.google.com/github/vivekverma1019/Handwritten-Word-Recognition/blob/master/tmp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Preprocessor.py

In [0]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

def add_padding(img, old_w, old_h, new_w, new_h):
    h1, h2 = int((new_h-old_h)/2), int((new_h-old_h)/2)+old_h
    w1, w2 = int((new_w-old_w)/2), int((new_w-old_w)/2)+old_w
    img_pad = np.ones([new_h, new_w, 3]) * 255
    img_pad[h1:h2, w1:w2,:] = img
    return img_pad

def fix_size(img, target_w, target_h):
    h, w = img.shape[:2]
    if w<target_w and h<target_h:
        img = add_padding(img, w, h, target_w, target_h)
    elif w>=target_w and h<target_h:
        new_w = target_w
        new_h = int(h*new_w/w)
        new_img = cv.resize(img, (new_w, new_h), interpolation = cv.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    elif w<target_w and h>=target_h:
        new_h = target_h
        new_w = int(w*new_h/h)
        new_img = cv.resize(img, (new_w, new_h), interpolation = cv.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    else:
        '''w>=target_w and h>=target_h '''
        ratio = max(w/target_w, h/target_h)
        new_w = max(min(target_w, int(w / ratio)), 1)
        new_h = max(min(target_h, int(h / ratio)), 1)
        new_img = cv.resize(img, (new_w, new_h), interpolation = cv.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    return img

def preprocess(path, img_w, img_h):
    """ Pre-processing image for predicting """
    img = cv.imread(path)
    img = fix_size(img, img_w, img_h)

    img = np.clip(img, 0, 255)
    img = np.uint8(img)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    img = img.astype(np.float32)
    img /= 255
    return img

### Paremeter.py

In [0]:
letters = [' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_classes = len(letters) + 1
img_h = 64

### Utils.py

In [0]:
import os
import numpy as np
import itertools
#from Parameter import *
#from Preprocessor import preprocess
from keras import backend as K


def decode_label(out):
    out_best = list(np.argmax(out[0, 2:], 1))
    out_best = [k for k, g in itertools.groupby(out_best)]
    outstr = ''
    for c in out_best:
        if c < len(letters):
            outstr += letters[c]
    return outstr

def decode_batch(out):
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = ''
        for c in out_best:
            if c < len(letters):
                outstr += letters[c]
        ret.append(outstr)
    return ret

def get_paths_and_texts(is_words):
    paths_and_texts = []
    if is_words:
        with open('/content/drive/My Drive/Downloads/iam_words/words.txt') as f:
            for line in f:
                if not line or line.startswith('#'):
                    continue
                line_split = line.strip().split(' ')
                assert len(line_split) >= 9
                status = line_split[1]
                if status == 'err':
                    continue
                file_name_split = line_split[0].split('-')
                label_dir = file_name_split[0]
                sub_label_dir = '{}-{}'.format(file_name_split[0], file_name_split[1])
                fn = '{}.png'.format(line_split[0])
                img_path = os.path.join('/content/drive/My Drive/Downloads/iam_words/words', label_dir, sub_label_dir, fn)
                gt_text = ' '.join(line_split[8:])
                paths_and_texts.append([img_path, gt_text])
    else:
        with open('../IAM_lines/lines.txt') as f:
            for line in f:
                if not line or line.startswith('#'):
                    continue
                line_split = line.strip().split(' ')
                assert len(line_split) >= 9
                status = line_split[1]
                if status == 'err':
                    continue
                file_name_split = line_split[0].split('-')
                label_dir = file_name_split[0]
                sub_label_dir = '{}-{}'.format(file_name_split[0], file_name_split[1])
                fn = '{}.png'.format(line_split[0])
                img_path = os.path.join('../IAM_lines/lines', label_dir, sub_label_dir, fn)
                gt_text = ' '.join(line_split[8:])
                gt_text = gt_text.replace('|', ' ')
                l = len(gt_text)
                if l<10 or l>74:
                    continue
                paths_and_texts.append([img_path, gt_text])
    return paths_and_texts

def predict_image(model_predict, path, is_word):
    if is_word:
        width = 128
    else:
        width = 800
    img = preprocess(path, width, 64)
    img = img.T
    if K.image_data_format() == 'channels_first':
        img = np.expand_dims(img, 0)
    else:
        img = np.expand_dims(img, -1)
    img = np.expand_dims(img, 0)

    net_out_value = model_predict.predict(img)
    pred_texts = decode_label(net_out_value)
    return pred_texts

Using TensorFlow backend.


### CRNN_Model.py

In [0]:
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation, Reshape, Lambda, BatchNormalization
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.layers.merge import add, concatenate

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def word_model():
    img_w = 128
    img_h = 64
    max_text_len = 16
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)

    # Make Networkw
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')  # (None, 128, 64, 1)

    # Convolution layer (VGG)
    inner = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)  # (None, 128, 64, 64)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)  # (None,64, 32, 64)

    inner = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)  # (None, 64, 32, 128)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)  # (None, 32, 16, 128)

    inner = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)  # (None, 32, 8, 256)

    inner = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(512, (3, 3), padding='same', name='conv6')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max4')(inner)  # (None, 32, 4, 512)

    inner = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(inner)  # (None, 32, 4, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)

    # CNN to RNN
    inner = Reshape(target_shape=((32, 2048)), name='reshape')(inner)  # (None, 32, 2048)
    inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)  # (None, 32, 64)

    # RNN layer
    gru_1 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)  # (None, 32, 512)
    gru_1b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
    reversed_gru_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_1b)

    gru1_merged = add([gru_1, reversed_gru_1b])  # (None, 32, 512)
    gru1_merged = BatchNormalization()(gru1_merged)
    
    gru_2 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
    gru_2b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
    reversed_gru_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_2b)

    gru2_merged = concatenate([gru_2, reversed_gru_2b])  # (None, 32, 1024)
    gru2_merged = BatchNormalization()(gru2_merged)

    # transforms RNN output to character activations:
    inner = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(gru2_merged) #(None, 32, 80)
    y_pred = Activation('softmax', name='softmax')(inner)

    labels = Input(name='the_labels', shape=[max_text_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    # loss function
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [y_pred, labels, input_length, label_length]
    )

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    model_predict = Model(inputs=input_data, outputs=y_pred)
    model_predict.summary()

    return model, model_predict


def line_model():
    img_w = 800
    max_text_len = 74
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)

    # Make Networkw
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')  # (None, 800, 64, 1)

    # Convolution layer
    inner = Conv2D(64, (5, 5), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)  # (None, 800, 64, 64)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)  # (None,400, 32, 64)

    inner = Conv2D(128, (5, 5), padding='same', name='conv2', kernel_initializer='he_normal')(inner)  # (None, 400, 32, 128)
    inner = Activation('relu')(inner)

    inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)  # (None, 400, 32, 128)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)  # (None, 200, 16, 128)
    
    inner = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(inner)  # (None, 200, 16, 256)
    inner = Activation('relu')(inner)
    
    inner = Conv2D(256, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(inner)  # (None, 200, 16, 256)
    inner = Activation('relu')(inner)
    
    inner = Conv2D(512, (3, 3), padding='same', name='conv6', kernel_initializer='he_normal')(inner)  # (None, 200, 16, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    
    inner = Conv2D(512, (3, 3), padding='same', name='conv7', kernel_initializer='he_normal')(inner)  # (None, 200, 16, 512)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max3')(inner)  # (None, 100, 8, 512)

    # CNN to RNN
    inner = Reshape(target_shape=((100, 4096)), name='reshape')(inner)  # (None, 100, 4096)
    inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)  # (None, 100, 64)

    # RNN layer
    gru_1 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)  # (None, 100, 512)
    gru_1b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
    reversed_gru_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_1b)

    gru1_merged = add([gru_1, reversed_gru_1b])  # (None, 100, 512)
    gru1_merged = BatchNormalization()(gru1_merged)
    
    gru_2 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
    gru_2b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
    reversed_gru_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_2b)

    gru2_merged = concatenate([gru_2, reversed_gru_2b])  # (None, 100, 1024)
    gru2_merged = BatchNormalization()(gru2_merged)

    # transforms RNN output to character activations:
    inner = Dense(80, kernel_initializer='he_normal',name='dense2')(gru2_merged) #(None, 100, 80)
    y_pred = Activation('softmax', name='softmax')(inner)

    labels = Input(name='the_labels', shape=[max_text_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    # loss function
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [y_pred, labels, input_length, label_length]
    )

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    model_predict = Model(inputs=input_data, outputs=y_pred)
    model_predict.summary()

    return model, model_predict

### ImageGenerator.py

In [0]:
import numpy as np
import random
from keras import backend as K
#from Preprocessor import preprocess
#from Parameter import *


def labels_to_text(labels):
    return ''.join(list(map(lambda x: letters[int(x)], labels)))

def text_to_labels(text):
    return list(map(lambda x: letters.index(x), text))

class TextImageGenerator:
    
    def __init__(self, data,
                 img_w,
                 img_h, 
                 batch_size, 
                 i_len,
                 max_text_len):
        
        self.img_h = img_h
        self.img_w = img_w
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.samples = data
        self.n = len(self.samples)
        self.i_len = i_len
        self.indexes = list(range(self.n))
        self.cur_index = 0
        
    def build_data(self):
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []
        for i, (img_filepath, text) in enumerate(self.samples):
            img = preprocess(img_filepath, self.img_w, self.img_h)
            self.imgs[i, :, :] = img
            self.texts.append(text)
    
    def next_sample(self):
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]
    
    def next_batch(self):
        while True:
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            if K.image_data_format() == 'channels_first':
                X_data = np.ones([self.batch_size, 1, self.img_w, self.img_h])
            else:
                X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])
            Y_data = np.zeros([self.batch_size, self.max_text_len])
            input_length = np.ones((self.batch_size, 1)) * self.i_len
            label_length = np.zeros((self.batch_size, 1))
                                   
            for i in range(self.batch_size):
                img, text = self.next_sample()
                img = img.T
                if K.image_data_format() == 'channels_first':
                    img = np.expand_dims(img, 0)
                else:
                    img = np.expand_dims(img, -1)
                X_data[i] = img
                Y_data[i, :len(text)] = text_to_labels(text)
                label_length[i] = len(text)
                
            inputs = {
                'the_input': X_data,
                'the_labels': Y_data,
                'input_length': input_length,
                'label_length': label_length,
            }
            outputs = {'ctc': np.zeros([self.batch_size])}
            yield (inputs, outputs)

### Train.py

In [0]:
#from Parameter import *
#from ImageGenerator import TextImageGenerator
#from CRNN_Model import word_model, line_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
#from Utils import *


def train(train_list, val_list, is_word_model):
    if is_word_model:
        input_length = 32 - 2
        model, _ = word_model()
        model_name = 'iam_words'
        max_text_len = 16
    else:
        input_length = 100 - 2
        model, _ = line_model()
        model_name = 'iam_line'
        max_text_len = 74

    batch_size = 32
    img_w = 128
    img_h = 64
    train_set = TextImageGenerator(train_list, img_w, img_h, batch_size, input_length, max_text_len)
    print('Loading data for train ...')
    train_set.build_data()
    val_set = TextImageGenerator(val_list, img_w, img_h, batch_size, input_length, max_text_len)
    val_set.build_data()
    print('Done')
    
    no_train_set = train_set.n
    no_val_set = val_set.n
    print("Number train set: ", no_train_set)
    print("Number val set: ", no_val_set)
    
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

    ckp = ModelCheckpoint(
        filepath=model_name+'--{epoch:02d}--{val_loss:.3f}.h5', monitor='val_loss',
        verbose=1, save_best_only=True, save_weights_only=True
    )
    earlystop = EarlyStopping(
        monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'
    )

    model.fit_generator(generator=train_set.next_batch(),
                        steps_per_epoch=no_train_set // batch_size,
                        epochs=32,
                        validation_data=val_set.next_batch(),
                        validation_steps=no_val_set // batch_size,
                        callbacks=[ckp, earlystop])

    return model

In [0]:

if __name__=='__main__':
    paths_and_texts = get_paths_and_texts()
    print('number of image: ', len(paths_and_texts))

    paths_and_texts_train, paths_and_texts_test = train_test_split(paths_and_texts, test_size=0.4, random_state=1707)
    paths_and_texts_val, paths_and_texts_test = train_test_split(paths_and_texts_test, test_size=0.5, random_state=1707)
    print('number of train image: ', len(paths_and_texts_train))
    print('number of valid image: ', len(paths_and_texts_val))
    print('number of test image: ', len(paths_and_texts_test))

    model = train(paths_and_texts_train, paths_and_texts_val, True)

In [0]:
paths_and_texts = get_paths_and_texts(True)
print('number of image: ', len(paths_and_texts))
paths_and_texts_train, paths_and_texts_test = train_test_split(paths_and_texts, test_size=0.4, random_state=1707)
paths_and_texts_val, paths_and_texts_test = train_test_split(paths_and_texts_test, test_size=0.5, random_state=1707)
print('number of train image: ', len(paths_and_texts_train))
print('number of valid image: ', len(paths_and_texts_val))
print('number of test image: ', len(paths_and_texts_test))
model = train(paths_and_texts_train, paths_and_texts_val, True)

number of image:  96456
number of train image:  57873
number of valid image:  19291
number of test image:  19292
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 128, 64, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 128, 64, 64)  0           batch_normali

In [0]:
import keras 
loaded_model = keras.models.load_model('/content/drive/My Drive/keras_model.h5')

In [0]:
def predict_image(model_predict, path, is_word):
    if is_word:
        width = 128
    else:
        width = 800
    img = preprocess(path, width, 64)
    img = img.T
    if K.image_data_format() == 'channels_first':
        img = np.expand_dims(img, 0)
    else:
        img = np.expand_dims(img, -1)
    img = np.expand_dims(img, 0)

    net_out_value = model_predict.predict(img)
    pred_texts = decode_label(net_out_value)
    return pred_texts

[<tf.Tensor 'softmax/truediv:0' shape=(?, 32, 80) dtype=float32>]

In [0]:
img = preprocess('/content/drive/My Drive/Downloads/iam_words/words/a01/a01-000u/a01-000u-00-01.png',128,64)
type(img), img.shape

(numpy.ndarray, (64, 128))

In [0]:
img = img.T
img.shape

(128, 64)

In [0]:
img = np.expand_dims(img,-1)
img.shape

(128, 64, 1)

In [0]:
img = np.expand_dims(img , 0)
img.shape

(1, 128, 64, 1)

In [0]:
net_out_value = loaded_model.predict(img)

In [0]:
net_out_value.shape

(1, 32, 80)

In [0]:
net_out_value

array([[[1.56408521e-06, 5.57086288e-09, 3.89102723e-08, ...,
         3.66089878e-08, 1.51185219e-07, 5.32454109e-17],
        [8.37857669e-07, 3.03258374e-09, 1.65351519e-08, ...,
         2.10945945e-08, 7.56529559e-08, 2.46695897e-17],
        [2.65278845e-06, 1.63754454e-10, 2.19709095e-09, ...,
         8.46826220e-10, 1.12611644e-08, 5.67723251e-16],
        ...,
        [3.09416892e-09, 6.91880220e-10, 1.61078805e-13, ...,
         5.87913846e-14, 1.36801473e-10, 9.99930978e-01],
        [1.33054954e-07, 1.01938431e-08, 4.09885190e-12, ...,
         1.02376302e-11, 6.65892674e-10, 9.55144167e-01],
        [2.76119209e-07, 1.47372461e-07, 7.40479233e-10, ...,
         1.95159133e-09, 2.03374739e-09, 6.53039024e-04]]], dtype=float32)

In [0]:
import h5py    
import numpy as np    
f1 = h5py.File('/content/drive/My Drive/keras_model.h5','r')

In [0]:
list(f1.keys())

['model_weights']

In [0]:
list(f1['model_weights'])

['activation_1',
 'activation_2',
 'activation_3',
 'activation_4',
 'activation_5',
 'activation_6',
 'activation_7',
 'add_1',
 'batch_normalization_1',
 'batch_normalization_2',
 'batch_normalization_3',
 'batch_normalization_4',
 'batch_normalization_5',
 'batch_normalization_6',
 'batch_normalization_7',
 'batch_normalization_8',
 'batch_normalization_9',
 'con7',
 'concatenate_1',
 'conv1',
 'conv2',
 'conv3',
 'conv4',
 'conv5',
 'conv6',
 'dense1',
 'dense2',
 'gru1',
 'gru1_b',
 'gru2',
 'gru2_b',
 'lambda_1',
 'lambda_2',
 'max1',
 'max2',
 'max3',
 'max4',
 'reshape',
 'softmax',
 'the_input']

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file("/content/drive/My Drive/keras_model.h5")
tflite_model = converter.convert()
open("/content/drive/My Drive/keras_model.tflite", "wb").write(tflite_model)

In [0]:
from keras.models import model_from_json
with open('/content/drive/My Drive/Downloads/word_model_predict.json', 'r') as f:
		model = model_from_json(f.read())

In [0]:
_, model = word_model()

In [0]:
model.load_weights('/content/drive/My Drive/Downloads/iam_words--15--1.791.h5')

In [0]:
predict_image(model, '/content/drive/My Drive/Downloads/iam_words/words/a01/a01-000u/a01-000u-00-01.png', is_word=True)

'MOVE'

In [0]:
loaded_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 64, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 64, 64)  0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
import tensorflow as tf
keras_file = "/content/drive/My Drive/keras_model.h5"
model.save(keras_file)

In [0]:
loaded_model.layers[1].get_weights()[0][0].shape

(3, 1, 64)

In [0]:
import keras
loaded_model = keras.models.load_model('/content/drive/My Drive/keras_model.h5')

In [0]:
loaded_model.summary()

In [0]:
predict_image(loaded_model, '/content/drive/My Drive/Downloads/iam_words/words/a01/a01-000u/a01-000u-01-06.png', is_word=True)

NameError: ignored

In [0]:
loaded_model.summary()

In [0]:
output_names = [node.op.name for node in loaded_model.inputs]

In [0]:
loaded_model.inputs[0].op.name

'the_input'

In [0]:
[node.op.name for node in loaded_model.inputs]

In [0]:
output_names

['the_input']

In [0]:
m_output_names = [node.op.name for node in model.outputs]
m_output_names

['softmax_1/truediv']

In [0]:
m_sess = tf.keras.backend.get_session()

In [0]:
m_frozen_def = tf.graph_util.convert_variables_to_constants(m_sess, m_sess.graph_def, m_output_names)

In [0]:
import tensorflow as tf
sess = tf.keras.backend.get_session()
sess

In [0]:
frozen_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_names)